## imports

In [84]:
import requests
import json
import jwt
import base64
from itertools import product
import re
from jsonschema import Draft202012Validator
import pandas as pd

## validators

In [85]:

specficationSchema = {
    "type": "object",
                 "patternProperties": {"^[a-zA-Z_][a-zA-Z0-9_]*$": {"type": "string"}}
                 }
varientsSchema = {
    "type": "object",
                 "properties": {
                                "color": {"type": "array", "minItems": 1, "items": {"type": "string"}},
                                "size": {"type": "array", "minItems": 1, "items": {"type": "string"}},                              
                                },
                                "required": ["color","size"],
                                "additionalProperties": False   
                 }

inventorySchema ={
            "type": "array",
            "minItems": 1,
            "items": {
              "type": "object",
              "properties": {
                "active": {"type": "boolean"}
              },
              "required": ["active"],
              "patternProperties": {
                "^[^\\s][^\\s]*$": {
                  "type": ["string", "boolean"]
                }
              },
              "additionalProperties": False
            }
          }

variation_overrideSchema = {"type": "array"}
locationIdSchema = {"type": "string"}
skuSchema = {"type": "string"}
ownerSchema = {"type": "object"}
providerSchema = {"type": "object","properties": {
                                "id": {"type": "string"},
                                "name": {"type": "string"},                              
                                },
                                "required": ["id","name"],
                                "additionalProperties": False }
sub_categorySchema = {"type": "string"}
cancellableSchema = {"type": "string"}
super_categorySchema = {"type": "string"}
sub_sub_categorySchema = {"type": "string"}
return_priceSchema = {"type": "number"}
mrpSchema = {"type": "number"}
long_descSchema = {"type": "string"}
ondc_category_idSchema = {"type": "string"}
returnableSchema  =  {"type": "string"}
ondc_statusSchema = {"type": "string"}
ondc_categorySchema = {"type": "string"}
sub_sub_category_idSchema = {"type": "string"}
short_descSchema = {"type": "string"}
categorySchema = {"type": "string"}
activeSchema = {
                               "type": "object","properties": {
                                "status": {"type": "string"},
                                "timeStamp": {"type": "string"},                              
                                },
                                "required": ["status","timeStamp"],
                                "additionalProperties": False
}
imagesSchema = {
    "type": "array",
    "minItems": 1,
    "items": {
        "type": "string",
        "pattern": "^(?!.*\.webp$).*"
    }
}
replaceableSchema = {"type": "string"}
statusSchema = {"type": "string"}
codSchema = {"type": "string"}
order_min_valueSchema = {"type": "string"}
time_to_shipSchema = {"type": "string"}
ondc_priceSchema = {"type": "number"}
manuf_detailSchema = {"type": "string"}
prod_codeSchema = {"type": "string"}
packer_detailSchema = {"type": "string"}


specfications_validator = Draft202012Validator(specficationSchema)
varients_validator = Draft202012Validator(varientsSchema)
inventory_validator = Draft202012Validator(inventorySchema)
locationId_validator = Draft202012Validator(locationIdSchema)
variation_override_validator = Draft202012Validator(variation_overrideSchema)

sku_validator = Draft202012Validator(skuSchema)
owner_validator = Draft202012Validator(ownerSchema)
provider_validator = Draft202012Validator(providerSchema)
sub_category_validator = Draft202012Validator(sub_categorySchema)
cancellable_validator = Draft202012Validator(cancellableSchema)

super_category_validator = Draft202012Validator(super_categorySchema)
sub_sub_category_validator = Draft202012Validator(sub_sub_categorySchema)
return_price_validator = Draft202012Validator(return_priceSchema)
mrp_validator = Draft202012Validator(mrpSchema)
long_desc_validator = Draft202012Validator(long_descSchema)

ondc_category_id_validator = Draft202012Validator(ondc_category_idSchema)
returnable_validator = Draft202012Validator(returnableSchema)
ondc_status_validator = Draft202012Validator(ondc_statusSchema)
ondc_category_validator = Draft202012Validator(ondc_categorySchema)
sub_sub_category_validator = Draft202012Validator(sub_sub_category_idSchema)

short_desc_validator = Draft202012Validator(short_descSchema)
category_validator = Draft202012Validator(categorySchema)
active_validator = Draft202012Validator(activeSchema)
images_validator = Draft202012Validator(imagesSchema)
replaceable_validator = Draft202012Validator(replaceableSchema)

status_validator = Draft202012Validator(statusSchema)
cod_validator = Draft202012Validator(codSchema)
order_min_value_validator = Draft202012Validator(order_min_valueSchema)
ondc_price_validator = Draft202012Validator(ondc_priceSchema)
manuf_detail_validator = Draft202012Validator(manuf_detailSchema)

prod_code_validator = Draft202012Validator(prod_codeSchema)
packer_detail_validator = Draft202012Validator(packer_detailSchema)

# specfications_validator.is_valid({"name_1":"string h"})

## token decoder

In [70]:
def decode_jwt_without_secret(token):
    try:
        # Split the token into its parts
        header, payload, signature = token.split('.')

        # JWT payloads are base64url encoded, but they might not have the right length,
        # so we pad them with '=' until they do.
        missing_padding = len(payload) % 4
        if missing_padding != 0:
            payload += '=' * (4 - missing_padding)

        # Decode the base64url encoded header and payload
        decoded_header = base64.urlsafe_b64decode(header + '==').decode('utf-8')
        decoded_payload = base64.urlsafe_b64decode(payload + '==').decode('utf-8')

        return {
            'header': decoded_header,
            'payload': decoded_payload
        }
    except Exception as e:
        return {'error': str(e)}

## get catalog by user token and status

In [71]:

def getCatalog(token,status):
    url = f"https://asia-south1-prod-shopcircuit-seller-portal.cloudfunctions.net/api/catalog_range?startIndex=1&endIndex=10000&status={status}"
    payload = {}
    headers = {
    'authority': 'asia-south1-prod-shopcircuit-seller-portal.cloudfunctions.net',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9',
    'authorization': f'Bearer {token}',
    'if-none-match': 'W/"e7a2-+n8mrFBLKpk+whrNle/ioRtJPeA"',
    'origin': 'https://dashboard.shopcircuit.ai',
    'referer': 'https://dashboard.shopcircuit.ai/',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    return json.loads(response.text)
    

## report generate


In [95]:
target_seller_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VyIjp7ImlzX3Bob25lX3ZlcmlmaWVkIjp0cnVlLCJyb2xlIjoiVVNFUiIsInNpZ25hdHVyZSI6bnVsbCwiZGVzY3JpcHRpb24iOm51bGwsImNyZWF0ZWRfYXQiOiIyMDIzLTEyLTA4VDA2OjU0OjA0LjA3MFoiLCJnc3RpbiI6IjM2QUFCQ0cyODM4RzFaUSIsImJhbmsiOnsiaWZzY19jb2RlIjoiUFVOQjAwMjA3MDAiLCJhY2NvdW50X251bWJlciI6NDQzNzAwMjEwMDAwMDE1MywiYmVuZWZpY2lhcnlfbmFtZSI6IkdUTiBFbmdpbmVlcmluZyBJbmRpYSBMaW1pdGVkIiwiYWNjb3VudF90eXBlIjoiQ3VycmVudCIsImJyYW5jaF9hZGRyZXNzIjoiQmFuayBTdHJlZXQsIEh5ZGVyYWJhZCJ9LCJjb21wYW55X2xvZ28iOiJodHRwczovL2VuY3J5cHRlZC10Ym4wLmdzdGF0aWMuY29tL2ltYWdlcz9xPXRibjpBTmQ5R2NSQUhYUGx1cTZHdFRSUERJSFJ2NWtKUHk4NnVGanA1c083aGcmdXNxcD1DQVUiLCJpc19nc3Rpbl92ZXJpZmllZCI6dHJ1ZSwiY29tcGFueSI6IkdUTiBFbmdpbmVlcmluZyBJbmRpYSBMdGQiLCJjb21wYW55X3VybCI6bnVsbCwic3RvcmVfdGltaW5nIjp7ImNsb3NlIjoiMTg6MzAiLCJvcGVuIjoiOTozMCJ9LCJlbWFpbCI6ImRpZ2l0YWxAZ3Ruc29ja3MuY29tIiwiYWdyZWVtZW50Ijp0cnVlLCJ0YXQiOiJQNUQiLCJtb2JpbGUiOjg1MDAzNTIzOTMsImdwcyI6IjE3LjU1ODgyMjEsIDc4LjIwODMxNTg0OTk5OTk5IiwiYXZhaWxhYmlsdHkiOnsiZW5hYmxlIjp0cnVlLCJ0aW1lc3RhbXAiOiIyMDIzLTEyLTA4VDA2OjU0OjA0LjA3MFoifSwibmFtZSI6IkdUTiBFbmdpbmVlcmluZyBJbmRpYSBMdGQiLCJhZ3JlZW1lbnRfdGltZSI6IjIwMjMtMTItMDhUMDY6NTQ6MDQuMDcwWiIsInN0YXR1cyI6IkFDVElWRSIsInByb2ZpbGVfaW1nIjoiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL3Nob3AtY2lyY3VpdC11c2VyLWRhdGEvaW1hZ2UvcHJvZmlsZS1waWN0dXJlL0dUTiBFbmdpbmVlcmluZyBJbmRpYSBMdGQtZ0w2RUJIUXlGZzlFVHZ6Z2dvV0QvTG9nby5qcGciLCJvcmRlcl9taW5fdmFsdWUiOjUwMCwibG9jYXRpb25zIjpbeyJhZGRyZXNzIjp7ImNvdW50cnkiOiJJbmRpYSIsInBpbmNvZGUiOjUwMjMwNywic3RkX2NvZGUiOiIwODQ1NSIsImNpdHkiOiJTYW5nYXJlZGR5Iiwic3RyZWV0IjoiSXNuYXB1ciIsImRpc3RyaWN0IjoiU2FuZ2FyZWRkeSIsImxvY2FsaXR5IjoiU1kgbm8gNDI3LTQyOCwgQ0hJVEtVTCBWSUxMQUdFLCBDSElUS1VMIFZJTExBR0UsIFBBVEFOQ0hFUlUgIiwic3RhdGUiOiJUZWxhbmdhbmEifSwicGlja3VwX2xvY2F0aW9uIjoiV29yayIsImdwcyI6IjE3LjczOTksIDc4LjE0NzcyIiwidGltZSI6eyJzY2hlZHVsZSI6eyJob2xpZGF5cyI6W119LCJyYW5nZSI6eyJzdGFydCI6IiIsImVuZCI6IiJ9LCJsYWJlbCI6ImVuYWJsZSIsInRpbWVzdGFtcCI6IjIwMjQtMDMtMDdUMDk6NDI6NDcuODU1WiJ9LCJsb2NhdGlvbl9pZCI6IjhlOWM0NDAyYTY4MDQ4NTc5NWY1MmM3YzJmZDIzODk2In1dLCJ1cGRhdGVkX2F0IjoiMjAyNC0wMy0wN1QwOTo0MzoyMC41MzNaIiwiaWQiOiJnTDZFQkhReUZnOUVUdnpnZ29XRCIsImF2YXRhciI6ImFzc2V0cy9pbWFnZXMvYXZhdGFycy9tYWxlLTA3LmpwZyJ9LCJpYXQiOjE3MTI5Mjc1NTF9.PvG8oSxk48hzicaVJpYdnd2SdopQeaCPgapyP7o-_bs"
seller_catalog = getCatalog(target_seller_token,"draft")
userData = decode_jwt_without_secret(target_seller_token)
# print(userData)
validator_column = ['Id','SKU',
    'IsSpecificationValid',
    'IsVariantsValid',
    'IsInventoryValid',
    'IsLocationIdValid',
    'IsVariantsOverrideValid',
    'IsSKUValid',
    'IsOwnerValid',
    'IsProviderValid',
    'IsSubCategoryValid',
    'IsCancellableValid',
    'IsSuperCategoryValid',
    'IsSubSubCategoryValid',
    'IsReturnPriceValid',
    'IsMRPValid',
    'IsLongDescValid',
    'IsOndcCategoryIdValid',
    'IsReturnableValid',
    'IsOndcStatusValid',
    'IsOndcCategoryValid',
    'IsSubSubCategoryIdValid',
    'IsShortDescValid',
    'IsCategoryValid',
    'IsActiveValid',
    'IsImagesValid',
    'IsReplaceableValid',
    'IsStatusValid',
    'IsCODValid',
    'IsOrderMinValueValid',
    'IsOndcPriceValid',
    'IsManufDetailValid',
    'IsProdCodeValid',
    'IsPackerDetailValid'
]


# Create an empty DataFrame with specified columns
df = pd.DataFrame(columns=validator_column)

# Iterate through seller_catalog and add rows to DataFrame
for doc in seller_catalog:
    print(doc)
    new_row = {'Id': doc['id'], 
                'SKU': doc["data"]["sku_id"], 
    'IsSpecificationValid': specfications_validator.is_valid(doc["data"]["specifications"]) if "specifications" in doc["data"] else False,
    'IsVariantsValid': varients_validator.is_valid(doc["data"]["variants"]) if "variants" in doc["data"] else False,
    'IsInventoryValid': inventory_validator.is_valid(doc["data"]["inventory"]) if "inventory" in doc["data"] else False,
    'IsLocationIdValid': locationId_validator.is_valid(doc["data"]["location_id"]) if "location_id" in doc["data"] else False,
    'IsVariantsOverrideValid': variation_override_validator.is_valid(doc["data"]["variation_override"]) if "variation_override" in doc["data"] else False,
    'IsSKUValid': sku_validator.is_valid(doc["data"]["sku_id"]) if "sku_id" in doc["data"] else False,
    'IsOwnerValid': owner_validator.is_valid(doc["data"]["owner"]) if "owner" in doc["data"] else False,
    'IsProviderValid': provider_validator.is_valid(doc["data"]["provider"]) if "provider" in doc["data"] else False,
    'IsSubCategoryValid': sub_category_validator.is_valid(doc["data"]["sub_category"]) if "sub_category" in doc["data"] else False,
    'IsCancellableValid': cancellable_validator.is_valid(doc["data"]["cancellable"]) if "cancellable" in doc["data"] else False,
    'IsSuperCategoryValid': super_category_validator.is_valid(doc["data"]["super_category"]) if "super_category" in doc["data"] else False,
    'IsSubSubCategoryValid': sub_sub_category_validator.is_valid(doc["data"]["sub_sub_category"]) if "sub_sub_category" in doc["data"] else False,
    'IsReturnPriceValid': return_price_validator.is_valid(doc["data"]["return_price"]) if "return_price" in doc["data"] else False,
    'IsMRPValid': mrp_validator.is_valid(doc["data"]["mrp"]) if "mrp" in doc["data"] else False,
    'IsLongDescValid': long_desc_validator.is_valid(doc["data"]["long_desc"]) if "long_desc" in doc["data"] else False,
    'IsOndcCategoryIdValid': ondc_category_id_validator.is_valid(doc["data"]["ondc_category_id"]) if "ondc_category_id" in doc["data"] else False,
    'IsReturnableValid': returnable_validator.is_valid(doc["data"]["returnable"]) if "returnable" in doc["data"] else False,
    'IsOndcStatusValid': ondc_status_validator.is_valid(doc["data"]["ondc_status"]) if "ondc_status" in doc["data"] else False,
    'IsOndcCategoryValid': ondc_category_validator.is_valid(doc["data"]["ondc_category"]) if "ondc_category" in doc["data"] else False,
    'IsSubSubCategoryIdValid': sub_sub_category_validator.is_valid(doc["data"]["sub_sub_category_id"]) if "sub_sub_category_id" in doc["data"] else False,
    'IsShortDescValid': short_desc_validator.is_valid(doc["data"]["short_desc"]) if "short_desc" in doc["data"] else False,
    'IsCategoryValid': category_validator.is_valid(doc["data"]["category"]) if "category" in doc["data"] else False,
    'IsActiveValid': active_validator.is_valid(doc["data"]["active"]) if "active" in doc["data"] else False,
    'IsImagesValid': images_validator.is_valid(doc["data"]["images"]) if "images" in doc["data"] else False,
    'IsReplaceableValid': replaceable_validator.is_valid(doc["data"]["replaceable"]) if "replaceable" in doc["data"] else False,
    'IsStatusValid': status_validator.is_valid(doc["data"]["status"]) if "status" in doc["data"] else False,
    'IsCODValid': cod_validator.is_valid(doc["data"]["cod"]) if "cod" in doc["data"] else False,
    'IsOrderMinValueValid': order_min_value_validator.is_valid(doc["data"]["order_min_value"]) if "order_min_value" in doc["data"] else False,
    'IsOndcPriceValid': ondc_price_validator.is_valid(doc["data"]["ondc_price"]) if "ondc_price" in doc["data"] else False,
    'IsManufDetailValid': manuf_detail_validator.is_valid(doc["data"]["manuf_detail"]) if "manuf_detail" in doc["data"] else False,
    'IsProdCodeValid': prod_code_validator.is_valid(doc["data"]["prod_code"]) if "prod_code" in doc["data"] else False,
    'IsPackerDetailValid': packer_detail_validator.is_valid(doc["data"]["packer_detail"]) if "packer_detail" in doc["data"] else False
}

    
    df.loc[len(df)] = new_row


df.to_excel(f"temp-Report.xlsx", index=False)

{'id': '0P2e8sU3U9BQmx4u9R3n', 'data': {'owner': {'_firestore': {'projectId': 'prod-shopcircuit-seller-portal'}, '_path': {'segments': ['users', 'gL6EBHQyFg9ETvzggoWD']}, '_converter': {}}, 'provider': {'name': 'GTN Engineering India Ltd', 'id': 'gL6EBHQyFg9ETvzggoWD'}, 'super_category': 'Men Fashion', 'featuredImage': None, 'category': 'Accessories', 'sub_category': 'Socks', 'sub_sub_category': 'Socks', 'ondc_category': 'Socks', 'ondc_category_id': '0b88e7fe7ffd', 'location_id': None, 'sub_sub_category_id': 'WQDYV7UQ0DudtD69eUv4', 'variation_override': [], 'images': ['https://storage.googleapis.com/shop-circuit-catalog-images/GTN Engineering India Ltd-gL6EBHQyFg9ETvzggoWD/0e741e2f-a32f-43e6-96dc-c6d5d0f4b7e51701433803546Socks1.jpg', 'https://storage.googleapis.com/shop-circuit-catalog-images/GTN Engineering India Ltd-gL6EBHQyFg9ETvzggoWD/04fdc09a-ccfc-41ef-92da-2e60297289541701433803540Socks2.jpg', 'https://storage.googleapis.com/shop-circuit-catalog-images/GTN Engineering India Ltd-g

# testing shell 

In [78]:
from jsonschema import validate, ValidationError
schema = {
  "type": "array",
  "minItems": 1,
  "items": {
    "type": "object",
    "properties": {
      "active": {"type": "boolean"}
    },
    "required": ["active"],
    "patternProperties": {
      "^[^\\s][^\\s]*$": {
        "type": ["string", "boolean"]
      }
    },
    "additionalProperties": False
  }
}



# Example data
data = [
    {"3S": "1","active": True},
     {"3XK": "1","active": True}
]

# Validate the data against the schema
try:
    validate(instance=data, schema=schema)
    print("Validation successful.")
except ValidationError as e:
    print("Validation failed:", e.message)


Validation successful.
